# Data Loading

In [50]:
import pandas as pd
df = pd.read_csv("winemag-data-130k-v2.csv", index_col=0)

# Building Embedings from tf-idf

In [51]:
# changing '-' into '_' so that tfidf can capture these words
wine_descriptions = df['description'].str.replace('-', '_', regex=False)
display(wine_descriptions[:5].tolist())

["Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",
 "This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.",
 'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless_steel fermented.',
 'Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey_drizzled guava and mango giving way to a slightly astringent, semidry finish.',
 "Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy count

making stop words for tf-idf

In [52]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

my_additional_stop_words = {'wine', 'drink', 'bottle', 'flavor', 'taste', 'like', 'nose', 'palate', 'finish', 'aroma', 'notes', 'note', 'vineyard', 'shows', 'alongside', 'offers', 'feels'}
all_stop_words = list(ENGLISH_STOP_WORDS.union(my_additional_stop_words))

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from config import *

# Tworzenie macierzy TF-IDF
vectorizer = TfidfVectorizer(
    max_features=50000,
    min_df=MIN_WORD_OCCURENCE,
    stop_words=all_stop_words,
    max_df=0.3,
    token_pattern=r'(?u)\b[\w-]{2,}\b'
    #ngram_range=(1, 2) # test z bigramami
)
display(MIN_WORD_OCCURENCE)
tfidf_matrix = vectorizer.fit_transform(wine_descriptions)
terms = vectorizer.get_feature_names_out()
print(f"Wymiary macierzy TF-IDF: {tfidf_matrix.shape}")

svd = TruncatedSVD(n_components=DIMENSIONS, random_state=RANDOM_STATE)
embeddings = svd.fit_transform(tfidf_matrix)
print(f"Wymiary embeddingów: {embeddings.shape}")

1000

Wymiary macierzy TF-IDF: (129971, 471)


In [ ]:
terms[:50]

array(['10', '100', '15', '20', '2015', '2016', '2017', '2018', '2019',
       '2020', '2022', '2025', 'accent', 'accented', 'accents', 'acidic',
       'acidity', 'acids', 'add', 'adds', 'aftertaste', 'age', 'aged',
       'aging', 'alcohol', 'almond', 'ample', 'anise', 'appeal',
       'appealing', 'appellation', 'apple', 'apples', 'approachable',
       'apricot', 'aromatic', 'astringent', 'attractive', 'backed',
       'background', 'baked', 'baking', 'balance', 'balanced', 'barrel',
       'bean', 'beautiful', 'beautifully', 'beef', 'berries'],
      dtype=object)

# Saving Embeddings

## Slow way to save embeddings in csv

In [ ]:
col_names = [f"svd_{i}" for i in range(DIMENSIONS)]
df_embeddings = pd.DataFrame(embeddings, columns=col_names)
df_embeddings = df_embeddings.astype('float32')
df_embeddings.to_csv("embeddings.csv", index=False)

display(df_embeddings.head())

,svd_0,svd_1,svd_2,svd_3,svd_4,svd_5,svd_6,svd_7,svd_8,svd_9,...,svd_118,svd_119,svd_120,svd_121,svd_122,svd_123,svd_124,svd_125,svd_126,svd_127
0,0.170740,-0.138452,-0.129760,0.014889,-0.077782,-0.119063,0.009996,0.017281,-0.047251,0.069879,...,0.003510,0.087378,0.019227,0.007721,-0.062861,0.000750,-0.039001,-0.001031,0.054339,0.020683
1,0.352410,0.048456,0.320737,0.227974,0.013773,-0.045483,-0.016798,-0.141270,-0.160663,-0.117332,...,0.014473,-0.020645,0.037858,-0.040340,-0.057215,-0.019766,-0.030956,0.033828,-0.034843,0.023275
2,0.168693,-0.232176,-0.056226,-0.001989,0.042146,-0.025812,-0.143039,0.058226,0.010733,0.076991,...,0.025059,-0.055313,-0.071085,-0.029049,0.062856,-0.047380,0.068758,-0.000703,-0.045919,-0.003368
3,0.099455,-0.096735,-0.077514,-0.085568,-0.056589,0.013805,-0.024250,-0.004128,-0.006844,-0.041499,...,0.017982,-0.015518,-0.028480,-0.027812,0.030760,0.009811,0.080501,-0.008027,0.006575,0.039487
4,0.117113,0.042183,-0.029024,-0.028174,0.025643,0.080079,-0.231041,-0.033764,0.103534,-0.095430,...,-0.054369,-0.023534,-0.006653,-0.027461,0.095537,-0.023791,0.010264,0.045348,0.007836,0.054381


## Efficient way to safe embeddings

In [ ]:
import numpy as np

embeddings = embeddings.astype(np.float32, copy=False)
np.save("embeddings.npy", embeddings)

## Comparing embedding file sizes

In [ ]:
import os
import humanize

for p in ["embeddings.npy", "embeddings.csv"]:
    size = os.path.getsize(p)
    print(p, humanize.naturalsize(size, binary=False))

embeddings.npy 66.5 MB
embeddings.csv 202.7 MB


# Concatenating with df and saving

In [ ]:
df_embeddings = df_embeddings.reset_index(drop=True)
df = df.reset_index(drop=True)
df_final = pd.concat([df, df_embeddings], axis=1)
df_final.to_csv(EMBEDED_FILEPATH, index=False)

# Building Embedings from tf-idf using pipeline

In [ ]:
# Same thing but using pipeline
from sklearn.pipeline import Pipeline

# Create a pipeline with TfidfVectorizer and TruncatedSVD
pipeline = Pipeline([
  ('tfidf', TfidfVectorizer(max_features=50000, min_df=MIN_WORD_OCCURENCE, stop_words=all_stop_words, max_df=0.3, token_pattern=r'(?u)\b[\w-]{2,}\b')),
  ('pca', TruncatedSVD(n_components=DIMENSIONS, random_state=RANDOM_STATE))
])

# Fit and transform the data using the pipeline
svd_matrix = pipeline.fit_transform(wine_descriptions)
print("Shape of SVD matrix:", svd_matrix.shape)

Shape of SVD matrix: (129971, 128)


# Look for most "informative" words in clusters using K-NN to find how well tf-idf dealt with noise data (e.g this, that)

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

def get_top_words(k_final, num_words=10):
    kmeans_final = KMeans(n_clusters=k_final, random_state=RANDOM_STATE, n_init=10)
    predicted_labels = kmeans_final.fit_predict(embeddings)

    # korzystamy z predicted labels do znalezienia słów w tym celu użyje TfidfVectorizer bo w tej interpretacji mam macierz słów do występowania ich
    for i in range(k_final):
        cluster_mask = (predicted_labels == i)
        cluster_tfidf_matrix = tfidf_matrix[cluster_mask] # type: ignore

        # licze centroid (licząc średni dla i-tego klastra w cluster_tfidf_matrix)
        cluster_centroid = np.asarray(cluster_tfidf_matrix.mean(axis=0)).flatten()

        top_indices = cluster_centroid.argsort()[-num_words:][::-1]
        top_terms = [terms[idx] for idx in top_indices]

        print(f"\nCluster {i}:")
        print(f"  (Liczba dokumentów: {np.sum(cluster_mask)})")
        print(f"  Top {num_words} terminów: {', '.join(top_terms)}") # type: ignore

In [ ]:
get_top_words(20, 15)


Cluster 0:
  (Liczba dokumentów: 5428)
  Top 15 terminów: crisp, acidity, fruity, fresh, bright, ready, aftertaste, character, texture, lively, attractive, fruits, light, ripe, tight

Cluster 1:
  (Liczba dokumentów: 6019)
  Top 15 terminów: cherry, tannins, red, berry, black, spice, clove, licorice, dried, leather, herb, firm, opens, raspberry, pepper

Cluster 2:
  (Liczba dokumentów: 5615)
  Top 15 terminów: berry, plum, spice, red, wild, cherry, tannins, earthy, oak, good, spicy, tannic, fruits, bright, baked

Cluster 3:
  (Liczba dokumentów: 3796)
  Top 15 terminów: bit, cherry, tannins, sweet, black, good, oak, red, dry, plum, green, spice, tart, raspberry, ripe

Cluster 4:
  (Liczba dokumentów: 8379)
  Top 15 terminów: black, cherry, pepper, tannins, plum, dark, spice, chocolate, currant, licorice, red, ripe, dried, rich, bottling

Cluster 5:
  (Liczba dokumentów: 5276)
  Top 15 terminów: soft, ripe, acidity, fruity, tannins, red, ready, fruits, texture, attractive, rounded, smo